In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


from reader import get_prices
from matplotlib import pyplot as plt
plt.style.use('dark_background')
import pandas as pd
import math
from keras.losses import MAPE
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sklearn
from sklearn.cluster import KMeans
import numpy as np
import pprint
from scipy.special import expit, logit

# fix random seed for reproducibility
np.random.seed(7)



In [ ]:
# IMPORT AND CLEAN DATASET

prices = get_prices()
symbols = prices.columns

for stock in prices:
    nan_dirty = prices[stock].isnull().sum().sum()
    if nan_dirty > 2:
        prices = prices.drop(stock, axis=1)
    elif nan_dirty:
        prices[stock].fillna(prices[stock].mean())
symbols = prices.columns

In [ ]:
# CLUSTER COMPANIES TOGETHER BASED ON STOCK TREND
# FEATURE VECTOR USED: 5-DAY PERCENTAGE CHANGE

weekly_priceDiff = {}

symbol_count = 0
for symbol in symbols:
    stock = prices[symbol]
    step = 5
    temp_ls = []

    for a in range(0,len(stock),step):
        stock_diff = 0
        opening_price = stock[a]
        closing_price = None
        try:
            closing_price = stock[a+step-1]
        except:
            closing_price = stock[a+step-2]
        stock_diff = (closing_price - opening_price)/opening_price*100
        temp_ls.append(stock_diff)
        
    weekly_priceDiff[symbol] = temp_ls
       

weekly_priceDiff = pd.DataFrame(weekly_priceDiff)
weekly_priceDiff = weekly_priceDiff.fillna(0)
weekly_priceDiff = weekly_priceDiff.transpose()

In [ ]:
weekly_priceDiff.head()

In [ ]:
clusters_no = 200

kmeans = KMeans(n_clusters=clusters_no, random_state=2).fit(weekly_priceDiff)

In [ ]:
# len(kmeans.labels_)
# print(kmeans.labels_)

centers = np.zeros(clusters_no)
for i in range(clusters_no):
    centers[kmeans.labels_[i]] += 1
    
# print(centers)
clusters = {}
for cluster in range(clusters_no):
    clusters[cluster] = symbols[np.where(kmeans.labels_==cluster)]
clusters

In [ ]:
# kmeans.predict([weekly_priceDiff.transpose().AAPL])


# AAPL_cluster = kmeans.predict([weekly_priceDiff.transpose().AAPL])
# for symbol in symbols:
#     if kmeans.predict([weekly_priceDiff.transpose()[symbol]]) == AAPL_cluster:
#         print(symbol)

In [ ]:
# RANK CORRELATIONS IN SAME CLUSTER

corr_A = []
corr_B = []
corr_coeff = []
for c in clusters:
    symb = clusters[c]
    if len(symb) > 1:
        values = []
        for s in symb:
            values.append(prices[s].fillna(prices[s].mean()).to_list()) # TO CHECK! filling NaN with mean
        cluster_corr = np.corrcoef(values)
        for i in range(len(symb)):
            for j in range(i+1,len(symb)):
                corr_A.append(symb[i])
                corr_B.append(symb[j])
                corr_coeff.append(cluster_corr[i][j])


d = {}
d["stock_A"] = corr_A
d["stock_B"] = corr_B
d["corr"] = corr_coeff
correlations = pd.DataFrame(data=d)

correlations = correlations.sort_values(["corr"], ascending=[0])
correlations = correlations.reset_index()
correlations = correlations.drop("index", axis=1)

In [ ]:
print(f"Candidate pairs found: {len(correlations)}")
plt.plot(correlations["corr"].to_list())

correlation_bins = (0,0,0,0,0)
bin0 = 0
bin1 = 0
bin2 = 0
bin3 = 0
bin4 = 0
bin5 = 0
for i in range(len(correlations)):
    corr = correlations.loc[i,"corr"]

    if corr >= 0.90:
        bin0+=1
    elif corr >= 0.75:
        bin1 += 1
    elif corr >= 0.50:
        bin2 += 1
    elif corr > -0.50:
        bin3 += 1
    elif corr > -0.75:
        bin4 += 1
    else:
        bin5 += 1
        
print(f" 1.00 -  0.90 Correlation pairs\t{bin0}\t({round(bin0/len(correlations)*100, 2)}%)")
print(f" 0.90 -  0.75 Correlation pairs\t{bin1}\t({round(bin1/len(correlations)*100, 2)}%)")
print(f" 0.75 -  0.50 Correlation pairs\t{bin2}\t({round(bin2/len(correlations)*100, 2)}%)")
print(f" 0.50 - -0.50 Correlation pairs\t{bin3}\t({round(bin3/len(correlations)*100, 2)}%)")
print(f"-0.50 - -0.75 Correlation pairs\t{bin4}\t({round(bin4/len(correlations)*100, 2)}%)")
print(f"-0.75 - -1.00 Correlation pairs\t{bin5}\t({round(bin5/len(correlations)*100, 2)}%)")


print('\n')
print(correlations)


In [ ]:
best_corr = correlations.loc[6,:]

# REGULAR PLOT
plt.plot(prices[best_corr.stock_A], label=best_corr.stock_A)
plt.plot(prices[best_corr.stock_B], label=best_corr.stock_B)
plt.legend()

# SCALED PLOT
# scaler = MinMaxScaler(feature_range=(0, 1))
# plt.plot(scaler.fit_transform(prices[best_corr.stock_A].to_numpy().reshape(-1,1)), label=best_corr.stock_A)
# plt.plot(scaler.fit_transform(prices[best_corr.stock_B].to_numpy().reshape(-1,1)), label=best_corr.stock_B)
# plt.legend()

best_corr

In [ ]:
# Retain model only for stock pairs w/ correlation above treshold
CORR_TRESHOLD = 0.95
filtered_pairs = pd.DataFrame([pair for _,pair in correlations.iterrows() if pair['corr'] >= CORR_TRESHOLD])
print(f"Filtered {len(filtered_pairs)} stock pairs with correlation above {CORR_TRESHOLD}")
filtered_pairs = filtered_pairs[6:7]
filtered_pairs

In [ ]:
# HELPER FUNCTION
# convert an array of values into a dataset matrix
look_back = 100
def xy_split_dataset(dataset, look_back=1, normalize=True):
    dataset = dataset.reshape(-1, 1)
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     if normalize:
#         dataset = scaler.fit_transform(dataset) 
    
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    X = np.array(dataX)
    Y = np.array(dataY)
    
    return X,Y


In [ ]:
# PREPARE DATASET FOR TRAINING AND TESTING

# dataset x-shape: (n_pairs, n_samples, look_back, 2)
#         y-shape: (n_pairs, n_samples, 2)

n_pairs = len(filtered_pairs)
n_samples = len(prices.loc[:,best_corr.stock_B].to_numpy()) - look_back
n_timesteps = look_back
n_features = 2

def pandas_fill(arr):
    input_shape = arr.shape
    df = pd.DataFrame(arr)
    df.fillna(method='ffill', axis=0, inplace=True)
    out = df.to_numpy().reshape(input_shape)
    return out

print("Rearranging dataset...")
x_train = np.zeros(shape=(n_pairs, n_samples, look_back, 2), dtype=float)
y_train = np.zeros(shape=(n_pairs, n_samples, 2), dtype=float)
scalers = np.empty(shape=(n_pairs), dtype=MinMaxScaler)
printer = True
pair_idx = 0
for _,pair in filtered_pairs.iterrows():
    print("Pair {} of {}".format(pair_idx+1, len(filtered_pairs)), end="\r")
    scaler = MinMaxScaler(feature_range=(0, 1))
    stock_A = prices.loc[:,pair.stock_A].to_numpy()
    stock_B = prices.loc[:,pair.stock_B].to_numpy()
    
    stock_A = pandas_fill(stock_A)
    stock_B = pandas_fill(stock_B)
    
#     [stock_A, stock_B] = scaler.fit_transform([stock_A, stock_B])  
    stock_A = np.log(stock_A)
    stock_B = np.log(stock_B)
    stock_A = pandas_fill(stock_A)
    stock_B = pandas_fill(stock_B)
    if np.sum(np.isnan(stock_A)):
        print("found NAN - A")
    if np.sum(np.isnan(stock_B)):
        print("found NAN - B")
    x_stock_A, y_stock_A = xy_split_dataset(stock_A, 
                                            look_back=look_back)
    x_stock_B, y_stock_B = xy_split_dataset(stock_B, 
                                            look_back=look_back)
    
    for sample_idx in range(len(x_stock_A)):
        for ts_idx in range(len(x_stock_A[sample_idx])):
            x_train[pair_idx][sample_idx][ts_idx][0] = x_stock_A[sample_idx][ts_idx]
            x_train[pair_idx][sample_idx][ts_idx][1] = x_stock_B[sample_idx][ts_idx]
        y_train[pair_idx][sample_idx][0] = y_stock_A[sample_idx]
        y_train[pair_idx][sample_idx][1] = y_stock_B[sample_idx]
    
    # save scalers
    scalers[pair_idx] = scaler
    if printer:
        printer = False
        pass
    
    pair_idx += 1

print("\nDONE")

In [ ]:
print("Full X dataset", x_train.shape)
print("Full Y dataset", y_train.shape)
train_test_split_ratio = 0.8
trainX = x_train.reshape(n_pairs, n_samples, n_timesteps, n_features)
trainY = y_train.reshape(n_pairs, n_samples, n_features)

split_index = int(n_samples*train_test_split_ratio)
testX = trainX[:,split_index:]
trainX = trainX[:,:split_index]
testY = trainY[:,split_index:]
trainY = trainY[:,:split_index]

# train_x_scalers = []
# test_scalers = []
# for a in range(len(trainX[0])):
#     scaler_x = MinMaxScaler(feature_range=(0, 1))
#     trainX[0][a] = scaler_x.fit_transform(trainX[0][a])
#     trainY[0][a] = scaler_x.transform(trainY[0][a].reshape(-1,1))
# for a in range(len(testX[0])):
#     scaler_x = MinMaxScaler(feature_range=(0, 1))
#     testX[0][a] = scaler_x.fit_transform(testX[0][a])
    

print("trainX ", trainX.shape)
print("trainY ", trainY.shape)
print("testX ", testX.shape)
print("testY ", testY.shape)

In [ ]:
# DEFINE AND TRAIN PREDICTIVE LSTM
model = Sequential()
model.add(LSTM(150, input_shape=((trainX.shape[2], trainX.shape[3]))))
model.add(Dense(2))
model.compile(loss=MAPE, optimizer='adam')
print(model.summary())
model.fit(trainX[0], trainY[0], epochs=5, batch_size=10, verbose=1)

In [ ]:
predicted = model.predict(testX[0])

print("True     :", np.exp(testY[0][0]))
print("Predicted:", np.exp(predicted[0]))
print(testY.shape)
print(predicted.shape)

plt.plot(np.exp(testY)[0,:,0], color = 'red', label = 'Stock Price')
plt.plot(np.exp(predicted)[:,0], color = 'green', label = 'Predicted Stock Price')
# plt.gca().set_ylim(bottom=0)
plt.legend()

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mean_absolute_percentage_error(np.exp(testY[0,:,0]), np.exp(predicted[:,0]))

In [ ]:
plt.plot(np.exp(testY)[0,:,1], color = 'red', label = 'Stock Price')
plt.plot(np.exp(predicted)[:,1], color = 'green', label = 'Predicted Stock Price')
# plt.gca().set_ylim(bottom=0)
plt.legend()
mean_absolute_percentage_error(np.exp(testY[0,:,1]), np.exp(predicted[:,1]))

In [ ]:
# # TEST MODEL PREFORMANCE
# for pair in range(len(filtered_pairs)):
#     # make predictions
#     trainPredict = model.predict(trainX[pair])
#     testPredict = model.predict(testX[pair])
    
# #     print(testX[pair])
#     if np.sum(np.isnan(testX[pair])):
#         print("found NAN")
#     print(len(trainX[pair,:,0,1]))
#     print(trainX[pair].shape)
#     print(trainPredict.shape)
    
    
    
#     # invert predictions (de-scale)
#     scaler = scalers[pair]
# #     trainPredict = scaler.inverse_transform(trainPredict)
# #     trainY[pair] = scaler.inverse_transform([trainY[pair]])
# #     testPredict = scaler.inverse_transform(testPredict)
# #     testY[pair] = scaler.inverse_transform([testY[pair]])
#     # calculate root mean squared error
#     trainScore = math.sqrt(mean_squared_error(np.exp(trainY[pair]), np.exp(trainPredict)))
#     print('Train Score: %.2f RMSE' % (trainScore))
#     testScore = math.sqrt(mean_squared_error(np.exp(testY[pair]), np.exp(testPredict)))
#     print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# DEFINING INVESTING AGENT POLICY

In [ ]:
# TEST AGENT PERFORMANCE